# 5. SAVE DATA

In [18]:
cont_vocab.save('context.vocab')
np.save('context', cont_id)
np.save('genres', genr_labels)

In [132]:
"""
    Origin:
    https://github.com/dennybritz/tf-rnn/blob/master/sequence_example.ipynb
"""

def make_example(sequence, label):
    # The object we return
    ex = tf.train.SequenceExample()
    # A non-sequential feature of our example
    sequence_length = len(sequence)
    ex.context.feature['length'].int64_list.value.append(sequence_length)
    ex.context.feature['labels'].int64_list.value.extend(label)
    # This part of TF is not so verbose
    # and tutorials are rare, also serialized labels were serialized with different length

    # Reshaped a bit WildML-s tips and tricks
    # http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/
    ex.feature_lists\
        .feature_list['tokens']\
        .feature.add()\
        .int64_list.value.extend(sequence)
    
        
    return ex

def write_TFRecord(fname, sequences, labels):
    with open(fname + '.TFRecord', 'w') as fp:
        writer = tf.python_io.TFRecordWriter(fp.namee: 0)
        print('Sampling...')
        i = 0
        for sequence, label in zip(sequences, labels):
            
            ex = make_example(sequence, label)
            writer.write(ex.SerializeToString())
            
            if i%500 == 0: print('\r%d'%i, end='')
            if i%5000 == 0: print(ex)
            i+=1
        writer.close()
        print("\nWrote to {}".format(fp.name))        

Understanding protobufs:

http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/

https://indico.io/blog/tensorflow-data-inputs-part1-placeholders-protobufs-queues/

At some point serialization is required, but in the example I used it isn't, but after the example is made the whole example is searialized to string.
Very poorly documented, and unstable.
Reading out in automated batch is described in WildML however, when
[[1, 2, 3], [4, 5, 6, 7]] is fed as input it does not work as expected.

However when reading with queues (which should be **built** in graph before being **initialized**, **coordinated** to end the reading session gracefully), the `dequeue` operator cooperates well with the `tf.train.batch` function




In [129]:
ex = tf.train.SequenceExample()
print(ex.ListFields())
ex.context.feature['length'].int64_list.value.extend([1, 2, 3])
# THIS IS STILL SOME BLACK MAGIC THOUGH...
ex.context.feature['length'].int64_list.value
ex.feature_lists.feature_list['tokens'].feature.add().int64_list.value.extend([9, 9, 9, 9])
print(ex.ListFields())

[]
[(<google.protobuf.pyext._message.FieldDescriptor object at 0x7fd1f3f65790>, feature {
  key: "length"
  value {
    int64_list {
      value: 1
      value: 2
      value: 3
    }
  }
}
), (<google.protobuf.pyext._message.FieldDescriptor object at 0x7fd1f3f65190>, feature_list {
  key: "tokens"
  value {
    feature {
      int64_list {
        value: 9
        value: 9
        value: 9
        value: 9
      }
    }
  }
}
)]


In [130]:
cont_list = len(cont_id) * [None]
for i in range(len(cont_id)):
    cont_list[i] = np.trim_zeros(cont_id[i])
    if i%500 == 0: print('\r%d'%i, end='')
#cont_list

98500

In [114]:
np.trim_zeros(cont_id)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [115]:
len(genr_labels), len(cont_list)

(98660, 98660)

In [116]:
genr_labels

array([[False, False,  True, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       ..., 
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False]], dtype=bool)

In [133]:
write_TFRecord('cnn', cont_list, genr_labels)

Sampling...
0context {
  feature {
    key: "labels"
    value {
      int64_list {
        value: 0
        value: 0
        value: 1
        value: 1
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
      }
    }
  }
  feature {
    key: "length"
    value {
      int64_list {
        value: 22
      }
    }
  }
}
feature_lists {
  feature_list {
    key: "tokens"
    value {
      feature {
        int64_list {
          value: 53
          value: 17
          value: 112
          value: 18
          value: 915
          value: 0
          value: 0
          value: 14
          value: 3559
          value: 5763
          value: 21
          value: 366
          value: 87
          value: 3751
    

In [20]:
genr_labels.astype(float).dtype

dtype('float64')

In [33]:
genr_id.shape

(98660, 11)

In [1]:
import codecs
import tensorflow as tf
import numpy as np 

In [17]:
def parse_example(filename_queue):
    # Define how to parse the example
    
    reader = tf.TFRecordReader()
    _, example = reader.read(filename_queue)
    
    context_features = {
        'length': tf.FixedLenFeature([], dtype=tf.int64),
        'labels': tf.FixedLenFeature([], dtype=tf.int64)
    }
    sequence_features = {
        "tokens": tf.FixedLenSequenceFeature([], dtype=tf.int64),
    }
    context_parsed, sequence_parsed = tf.parse_single_sequence_example(
        serialized=example,
        context_features=context_features,
        sequence_features=sequence_features
    )
    return context_parsed, sequence_parsed
    

In [18]:
filename_queue = tf.train.string_input_producer(['cnn.TFRecord'])
parsed = parse_example(filename_queue)
parsed

({'labels': <tf.Tensor 'ParseSingleSequenceExample_7/ParseSingleSequenceExample:0' shape=() dtype=int64>,
  'length': <tf.Tensor 'ParseSingleSequenceExample_7/ParseSingleSequenceExample:1' shape=() dtype=int64>},
 {'tokens': <tf.Tensor 'ParseSingleSequenceExample_7/ParseSingleSequenceExample:2' shape=(?,) dtype=int64>})

Reading a single SeqExample

In [19]:
filename_queue = tf.train.string_input_producer(['cnn.TFRecord'])
parsed = parse_example(filename_queue)

with tf.Session() as sess:
    
    coord = tf.train.Coordinator()
    
    sess.run(tf.global_variables_initializer())
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    for i in range(3):
        print(i)
        res = sess.run(parsed)
        print(res)
    coord.request_stop()
    coord.join(threads)
    sess.close()

0
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Run call was cancelled


InvalidArgumentError: Name: , Key: labels, Index: 0.  Number of int64 values != expected.  values size: 24 but output shape: []
	 [[Node: ParseSingleSequenceExample_8/ParseSingleSequenceExample = ParseSingleSequenceExample[Ncontext_dense=2, Ncontext_sparse=0, Nfeature_list_dense=1, Nfeature_list_sparse=0, Tcontext_dense=[DT_INT64, DT_INT64], context_dense_shapes=[[], []], context_sparse_types=[], feature_list_dense_shapes=[[]], feature_list_dense_types=[DT_INT64], feature_list_sparse_types=[], _device="/job:localhost/replica:0/task:0/cpu:0"](ReaderRead_9:1, ParseSingleSequenceExample_8/ParseSingleSequenceExample/feature_list_dense_missing_assumed_empty, ParseSingleSequenceExample_8/ParseSingleSequenceExample/context_dense_keys_0, ParseSingleSequenceExample_8/ParseSingleSequenceExample/context_dense_keys_1, ParseSingleSequenceExample_8/ParseSingleSequenceExample/feature_list_dense_keys_0, ParseSingleSequenceExample_8/Const, ParseSingleSequenceExample_8/Const_1, ParseSingleSequenceExampl

This is driving me crazy

```
Name: , Key: tokens, Index: 0.  Number of int64 values != expected.  values size: 22 but output shape: []
```

however triple checked, that every single line in genre_labels are 24 length
still serialized to 22...

How to read batched SeqExamples

In [6]:
filename_queue = tf.train.string_input_producer(['cnn.TFRecord'])
sequence_parsed = parse_example(filename_queue)
batched_data = tf.train.batch(
        tensors=sequence_parsed,
        batch_size=2,
        dynamic_pad=True
    )

x = batched_data['tokens']
y = batched_data['labels']
res = []
with tf.Session() as sess:
    
    coord = tf.train.Coordinator()
    sess.run(tf.global_variables_initializer())
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    try:
        while not coord.should_stop():
            #pass
            res.append(x.eval().shape[1])
    except tf.errors.OutOfRangeError:
        print('Done training -- epoch limit reached')
    finally:
        # When done, ask the threads to stop.
        coord.request_stop()
    #res = sess.run(sequence_parsed['tokens'])
    #res = sess.run(sequence_parsed['tokens'])
    #res = batched_data.eval()
    #res1 = tf.contrib.learn.run_n({'y':batched_data})
    #res2 = tf.contrib.learn.run_n({'y':batched_data}, n=6)
    #print(res[0]['y'])
    coord.join(threads)
    sess.close()

SystemError: <built-in function TF_NewBuffer> returned a result with an error set

In [8]:
len(res)

14644

In [53]:
for run in res2:
    for line in run['y']:
        print(len(line))

22
22
13
13
10
10
7
7
10
10
3
3


In [46]:
print(res)
print(res1)
print(res2)

[  52   13  120   16  798    0    0   19 2999    0   18  349   95 3811    0
 1519  455   56   30    3    0  159]
[{'y': array([[  52,   13,  120,   16,  798,    0,    0,   19, 2999,    0,   18,
         349,   95, 3811,    0, 1519,  455,   56,   30,    3,    0,  159]])}]
[{'y': array([[  52,   13,  120,   16,  798,    0,    0,   19, 2999,    0,   18,
         349,   95, 3811,    0, 1519,  455,   56,   30,    3,    0,  159]])}]


In [32]:
res2

[{'y': array([[  52,   13,  120,   16,  798,    0,    0,   19, 2999,    0,   18,
           349,   95, 3811,    0, 1519,  455,   56,   30,    3,    0,  159]])}]

In [62]:
type(cont_id[:10].tolist()[0][0])

int

In [8]:
filename = "cnn.TFRecord"
for serialized_example in tf.python_io.tf_record_iterator(filename):
    example = serialized_example
    
    context_features = {
        "length" : tf.FixedLenFeature([1], dtype=tf.int64),
    }
    sequence_features = {
        "tokens": tf.FixedLenSequenceFeature([], dtype=tf.int64),
        "labels": tf.FixedLenSequenceFeature([], dtype=tf.int64)
    }
    context_parsed, sequence_parsed = tf.parse_single_sequence_example(
        serialized=example,
        context_features=context_features,
        sequence_features=sequence_features
    )
    
    # traverse the Example format to get data
    print(context_parsed, sequence_parsed)
    break

{'length': <tf.Tensor 'ParseSingleSequenceExample_2982/ParseSingleSequenceExample:0' shape=(1,) dtype=int64>} {'tokens': <tf.Tensor 'ParseSingleSequenceExample_2982/ParseSingleSequenceExample:2' shape=(?,) dtype=int64>, 'labels': <tf.Tensor 'ParseSingleSequenceExample_2982/ParseSingleSequenceExample:1' shape=(?,) dtype=int64>}


Sampling...
20000
40000
60000
80000
100000
120000
Wrote to encoder-decoder.TFRecord


Writing ./stride1-qa-16384.tfrecords


In [25]:
for q, a, g in zip(questions[1000:1010], answers[1000:1010], genres[1000:1010]):
    print(q, '<<--- ', g, '--->>', a, end='\n\n\n')

Why should I carry your bag?  I am not a dog. <<---  ['action', 'crime', 'drama', 'thriller'] --->> For five years I paid for your stupidness - you'll carry my bag for the rest of my life if I say so.  Unless you refuse, Oleg.


Turn that off!  Get the bags. <<---  ['action', 'crime', 'drama', 'thriller'] --->> Why should I carry your bag?  I am not a dog.


What? <<---  ['action', 'crime', 'drama', 'thriller'] --->> Smell like chemicals...for smoking drugs.


Turn that fucking thing off! <<---  ['action', 'crime', 'drama', 'thriller'] --->> I'm not filming.  I'm watching Milos die.  It's just like a move but realer.


You said speak Czech! <<---  ['action', 'crime', 'drama', 'thriller'] --->> How you erase this?


Speak English! <<---  ['action', 'crime', 'drama', 'thriller'] --->> You said speak Czech!


How you erase this? <<---  ['action', 'crime', 'drama', 'thriller'] --->> I'll do it.  Don't hurt my camera!


Whore? <<---  ['action', 'crime', 'drama', 'thriller'] --->> I'm homesi